# Chapter 9  
## Metaprogramming

One of the most important mantras of software development is "don’t repeat yourself."  
That is, any time you are faced with a problem of creating highly repetitive code (or cutting and pasting source code), it often pays to look for a more elegant solution.  
In Python, such problems are often solved under the category of "metaprogramming."  
In a nutshell, metaprogramming is about creating functions and classes whose main goal is to manipulate code by modifying, generating, or wrapping existing code.  
The main features for this include decorators, class decorators, and metaclasses.  
However, a variety of other useful topics -- including signature objects, execution of code with exec(), and inspecting the internals of functions and classes -- enter the picture.  
The main purpose of this chapter is to explore various metaprogramming techniques and to give examples of how they can be used to customize the behavior of Python to your own whims.

## 9.1 Putting a Wrapper Around a Function

### Problem

You want to put a wrapper layer around a function that adds extra processing, such as logging, timing, and so forth.

### Solution

You can define a decorator function to wrap a function with extra code.

In [6]:
import time
from functools import wraps

def timethis(func):
    """
    Decorator that reports the execution time.
    """
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result
    return wrapper

Now let's make our decorator useful.

In [7]:
@timethis
def countdown(n):
    """
    Yes, just a countdown.
    """
    while n > 0:
        n -= 1

In [8]:
countdown(10000)

countdown 0.0005571842193603516


In [9]:
countdown(100000)

countdown 0.009731054306030273


### Discussion

A decorator is a function that accepts a function as input and returns a new function as output.  
Whenever you write code like the following:

In [10]:
@timethis
def countdown(n):
    '''do something'''

it's the same as if you had executed these separate steps:

In [11]:
def countdown(n):
    '''...'''
countdown = timethis(countdown)

Python's built-in decorators, like `@staticmethod`, `@classmethod`, and `@property` work in the same way.  
For example, the following two code fragments will give an equivalent result:

In [12]:
class A:
    @classmethod
    def method(cls):
        pass
    
class B:
    # Equivalent definition of a class method.
    def method(cls):
        pass
    method = classmethod(method)

The code inside a decorator typically involves creating a new function that accepts any arguments using `*args` and `**kwargs`, as shown with the `wrapper()` function in this recipe.  
Inside this function, you place a call to the original input function and return its result.  
However, you also place whatever extra code you want to add, like our timing function.  
The newly created function wrapper is returned as a result and takes the place of the original function.  
It’s critical to emphasize that decorators generally do not alter the calling signature or return value of the function being wrapped.  
The use of `*args` and `**kwargs` is there to make sure that any input arguments can be accepted.  
The return value of a decorator is almost always the result of calling `func(*args, **kwargs)`, where `func` is the original unwrapped function.  
When first learning about decorators, it is usually very easy to get started with some simple examples, such as the one shown.  
However, if you are going to write decorators for real, there are some subtle details to consider.  
For example, the use of the decorator `@wraps(func)` in the solution is an easy to forget but important technicality related to preserving function metadata, which is described in the next recipe.  
The next few recipes that follow fill in some details that will be important if you wish to write decorator functions of your own.

## 9.2. Preserving Function Metadata When Writing Decorators